In [1]:
import ruptures as rpt  # our package
from ruptures.metrics import hausdorff
import pandas as pd
import concurrent.futures
import sys
from collections import defaultdict
from scipy.spatial import distance
import numpy as np

infile="/Users/irffanalahi/Research/Research_code/gitignorefolder/DMRrelated/ITGAE_patternrecognition/changepointtest/preproocess_Develop/preprocess_ontopmetilene/Again/betweenchildrencorr/BluOurPBLOurCD8til_wg_all_matrixCin_nr0.5_imputed_g1_CD8TIL_3_g2_others_36.txt_q0.00001_diff0.1_hypo.txt_subset_PELT_anomalyname3.txt" #sys.argv[1] 
minCpG=3 #int(sys.argv[2]) #3
penalty_value =3.5 #float(sys.argv[3]) #1 
percormetDMR=2#int(sys.argv[4])  #2 #### for parallal
outfile=infile+"_mincpg_"+str(minCpG)+"pelt.txt" #sys.argv[5] 
outsegmentfile=infile+"_mincpg_"+str(minCpG)+"pelt_seginfo.txt" #sys.argv[6]

indf=pd.read_csv(infile,sep="\t",low_memory=False,index_col=[0,1,2])

indf.head()

CD8TIL-CD4bluourPBL  CD8TIL-CD8bluourPBL  CD8TIL-nB  \
chrom start    end                                                             
chr1  17619076 17619078            -0.154000            -0.155800  -0.272333   
      17619107 17619109            -0.101400            -0.015000  -0.138667   
      17619157 17619159            -0.153933            -0.014333  -0.258000   
      17619187 17619189            -0.057800            -0.027400  -0.063333   
      17619191 17619193            -0.059667             0.036733  -0.121333   

                         CD8TIL-NK  CD8TIL-MnbluourPBL  CD8TIL-iDC  \
chrom start    end                                                   
chr1  17619076 17619078  -0.165333           -0.179200   -0.194333   
      17619107 17619109  -0.114000           -0.128400   -0.143667   
      17619157 17619159  -0.183667           -0.007733    0.009333   
      17619187 17619189  -0.073667            0.033600   -0.046000   
      17619191 17619193  -0.096333           -0.121867   -0.093333   

                         CD8TIL-mDC  CD8TIL-mNeu  CD8TIL-cB  CD8TIL-MEL_TUMOR  \
chrom start    end                                                              
chr1  17619076 17619078   -0.179667    -0.267000  -0.034500         -0.298333   
      17619107 17619109   -0.107333    -0.169000   0.046500         -0.125667   
      17619157 17619159   -0.076667    -0.273000   0.172167         -0.356333   
      17619187 17619189    0.004000    -0.053333   0.151000         -0.110000   
      17619191 17619193   -0.120667    -0.102333   0.119667         -0.154333   

                        DMRchrom  DMRstart    DMRend             q      diff  \
chrom start    end                                                             
chr1  17619076 17619078     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619107 17619109     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619157 17619159     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619187 17619189     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619191 17619193     chr1  17619077  17620436  3.301700e-07 -0.111284   

                         #cpg        p(MWU)       p(2dks)   CD8TIL  others  
chrom start    end                                                          
chr1  17619076 17619078    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619107 17619109    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619157 17619159    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619187 17619189    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619191 17619193    30  4.607400e-14  2.319700e-10  0.77251  0.8838

In [2]:

indfdropped=indf#.drop(['chrom','start','end'],axis=1)
columnnames=(indfdropped.columns).tolist()
indfdropped.head()

CD8TIL-CD4bluourPBL  CD8TIL-CD8bluourPBL  CD8TIL-nB  \
chrom start    end                                                             
chr1  17619076 17619078            -0.154000            -0.155800  -0.272333   
      17619107 17619109            -0.101400            -0.015000  -0.138667   
      17619157 17619159            -0.153933            -0.014333  -0.258000   
      17619187 17619189            -0.057800            -0.027400  -0.063333   
      17619191 17619193            -0.059667             0.036733  -0.121333   

                         CD8TIL-NK  CD8TIL-MnbluourPBL  CD8TIL-iDC  \
chrom start    end                                                   
chr1  17619076 17619078  -0.165333           -0.179200   -0.194333   
      17619107 17619109  -0.114000           -0.128400   -0.143667   
      17619157 17619159  -0.183667           -0.007733    0.009333   
      17619187 17619189  -0.073667            0.033600   -0.046000   
      17619191 17619193  -0.096333           -0.121867   -0.093333   

                         CD8TIL-mDC  CD8TIL-mNeu  CD8TIL-cB  CD8TIL-MEL_TUMOR  \
chrom start    end                                                              
chr1  17619076 17619078   -0.179667    -0.267000  -0.034500         -0.298333   
      17619107 17619109   -0.107333    -0.169000   0.046500         -0.125667   
      17619157 17619159   -0.076667    -0.273000   0.172167         -0.356333   
      17619187 17619189    0.004000    -0.053333   0.151000         -0.110000   
      17619191 17619193   -0.120667    -0.102333   0.119667         -0.154333   

                        DMRchrom  DMRstart    DMRend             q      diff  \
chrom start    end                                                             
chr1  17619076 17619078     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619107 17619109     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619157 17619159     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619187 17619189     chr1  17619077  17620436  3.301700e-07 -0.111284   
      17619191 17619193     chr1  17619077  17620436  3.301700e-07 -0.111284   

                         #cpg        p(MWU)       p(2dks)   CD8TIL  others  
chrom start    end                                                          
chr1  17619076 17619078    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619107 17619109    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619157 17619159    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619187 17619189    30  4.607400e-14  2.319700e-10  0.77251  0.8838  
      17619191 17619193    30  4.607400e-14  2.319700e-10  0.77251  0.8838

In [3]:
def segment_generator():
    
    segmentlist=[]
    
    qindex=columnnames.index('q')

    subdf=indfdropped.iloc[:,:qindex]
   
    grouped=subdf.groupby(['DMRchrom','DMRstart','DMRend'])
    for name, group in grouped:
        subgroup=group.drop(["DMRchrom","DMRstart","DMRend"],axis=1)
        segmentlist.append((name,subgroup))
    
  
    
    return segmentlist


In [4]:
class PELT_result:
    def __init__(self,underlyingdf,dmrchrom,dmrstart,dmrend,peltsegmentlist):
        self.underlyingdf=underlyingdf
        self.underlyingdf_index=self.underlyingdf.index
        
        
        self.dmrchrom=dmrchrom
        self.dmrstart=dmrstart
        self.dmrend=dmrend
        self.peltsegmentlist=peltsegmentlist
        self.fullDMRcondnumber=np.linalg.cond(self.underlyingdf,p=2)
        self.totalsegment=len(self.peltsegmentlist)-1
        self.metpeltsegmentDICT={'DMRchrom':self.dmrchrom,'DMRstart':self.dmrstart,'DMRend':self.dmrend,"totalPeltSegement":self.totalsegment,"conditionnumber":self.fullDMRcondnumber}
        self.metpeltsegmentDICTinfo=self.recordPeltDMR() #{'DMRchrom':self.dmrchrom,'DMRstart':self.dmrstart,'DMRend':self.dmrend,"PeltSegement":self.peltsegmentlist}
        
    def recordPeltDMR(self):
        underlyingdfavg=self.underlyingdf.mean(axis=0)
       
        underlyingdfmax=underlyingdfavg.max()
        underlyingdfmin=underlyingdfavg.min()
        
        
        
        prevcorressavgdiff=pd.Series(dtype='float64')
        
        outdict=defaultdict(list)
        nextstartloc=self.underlyingdf_index.get_loc((self.dmrchrom,self.dmrstart-1,self.dmrstart+1))
        prevend=self.dmrstart-1
        
       
        for peltsegment in self.peltsegmentlist:
            
            
            currentstartindex=self.underlyingdf_index[nextstartloc]
            nextstartloc=self.underlyingdf_index.get_loc(peltsegment)
            currentendindex=self.underlyingdf_index[nextstartloc-1]
            
            if peltsegment == self.peltsegmentlist[-1]:
                currentendindex=self.underlyingdf_index[nextstartloc]
            
           
            
            outdict['chrom'].append(currentstartindex[0])
            outdict['start'].append(currentstartindex[1]+1)
            outdict['end'].append(currentendindex[2])
            
            
          
            corressdf=self.underlyingdf.loc[currentstartindex:currentendindex, :]
            
          
           
            
            corresscondnumber=np.linalg.cond(corressdf,p=2)
            corressavgdiff=corressdf.mean(axis=0) 
            corressavgdict=corressavgdiff.to_dict()
            
            #####distances#########
            cosdistance=distance.cosine(corressavgdiff,underlyingdfavg)
            eucliddistance=distance.euclidean(corressavgdiff,underlyingdfavg)
            corrdistance=distance.correlation(corressavgdiff,underlyingdfavg)
            
            ####between children####
           
            if prevcorressavgdiff.empty:
                childrencosdistance=-99999
                childreneucliddistance=-99999
                childrencorrdistance=-99999
            else:
                childrencosdistance=distance.cosine(corressavgdiff,prevcorressavgdiff)
                childreneucliddistance=distance.euclidean(corressavgdiff,prevcorressavgdiff)
                childrencorrdistance=distance.correlation(corressavgdiff,prevcorressavgdiff)
            
            #####distances#########
            
            
            
            
            
            outdict['numofcpg'].append(corressdf.shape[0])
            
            
            
            outdict['conditionnumber'].append(corresscondnumber)
            outdict['parentDMRconditionnumber'].append(self.fullDMRcondnumber)
            outdict['conditionnumber/parentDMRconditionnumber'].append(corresscondnumber/self.fullDMRcondnumber)
            
            outdict['parentDMRchrom'].append(self.dmrchrom)
            outdict['parentDMRstart'].append(self.dmrstart)
            outdict['parentDMRend'].append(self.dmrend)
            
            outdict['parentnumofcpg'].append((self.underlyingdf).shape[0])
            
            
            
            
            outdict['cos_distance'].append(cosdistance)
            outdict['euclid_distance'].append(eucliddistance)
            outdict['correlation_distance'].append(corrdistance)
            
            
            outdict['prevchild_cos_distance'].append(childrencosdistance)
            outdict['prevchild_euclid_distance'].append(childreneucliddistance)
            outdict['prevchild_correlation_distance'].append(childrencorrdistance)
            
            for key, value in corressavgdict.items():
                outdict[key].append(value)
            
            #print(outdict)
            #sys.exit(1)
            outdict['parentDMRmax'].append(underlyingdfmax)
            outdict['parentDMRmin'].append(underlyingdfmin)
            
            prevend=peltsegment[2]
            
            prevcorressavgdiff=corressavgdiff
        
        outdict=dict(outdict)
        if self.peltsegmentlist[len(self.peltsegmentlist)-1][2]!=self.dmrend:
            print(self.dmrchrom,self.dmrstart,self.dmrend)
            print("final pelt seg notin DMR end")
        return outdict
            

In [5]:
def get_pelt_result(signal_pelttuple):
    DMR=signal_pelttuple[0]
    signal_pelt=signal_pelttuple[1]
    try:
        algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal_pelt)
        bkps_python = algo_python.predict(pen=penalty_value)
        forindexgenerate=bkps_python
        if forindexgenerate[-1]==len(signal_pelt.index):
            forindexgenerate[-1]=forindexgenerate[-1]-1
        pr=PELT_result(signal_pelt,DMR[0],DMR[1],DMR[2],(signal_pelt.index[forindexgenerate]).tolist())
    except: ### gives error if #cpg is smaller.thats why try-except commannd
        
        pr=PELT_result(signal_pelt,DMR[0],DMR[1],DMR[2],[signal_pelt.index[len(signal_pelt.index)-1]])
    return pr

def run_get_pelt_result(signalchunks):
    signalchunksresult=[]
    for signalchunk in signalchunks:
        signalchunksresult.append(get_pelt_result(signalchunk))
    return signalchunksresult

In [6]:
signallist=segment_generator()
chunks = [signallist[x:x+percormetDMR] for x in range(0, len(signallist), percormetDMR)]

print(len(chunks))

13


In [7]:
multiresult = []

with concurrent.futures.ProcessPoolExecutor() as executor:
    processlist=[]
    for chunk in chunks:
        processlist.append(executor.submit(run_get_pelt_result,chunk))
        #algo_python = rpt.Pelt(model="rbf", jump=1, min_size=minCpG).fit(signal)  # written in pure python
    for process in concurrent.futures.as_completed(processlist):
        multiresult=multiresult+process.result()


In [8]:

allresultdictlist=[]
allresultsegmentinfolist=defaultdict(list)
for resultobj in multiresult:
    allresultdictlist.append(resultobj.metpeltsegmentDICT)
    
    for key, value in resultobj.metpeltsegmentDICTinfo.items():
        allresultsegmentinfolist[key].append(value)
    
    
allresultdf=pd.DataFrame(allresultdictlist)
allresultsegmentinfolist=dict(allresultsegmentinfolist)

anotherdict={}
for key , value in allresultsegmentinfolist.items():
   
    anotherdict[key]=[x for sublist in value for x in sublist]
    
    
allresultsegmentinfolist=anotherdict

allresultsegmentinfodf=pd.DataFrame.from_dict(allresultsegmentinfolist)


diffcellcolumns=columnnames[:columnnames.index('DMRchrom')]

allresultsegmentinfodf['PELTmax']=allresultsegmentinfodf[diffcellcolumns].max(axis=1)
allresultsegmentinfodf['PELTmin']=allresultsegmentinfodf[diffcellcolumns].min(axis=1)


allresultsegmentinfodf['PELTmax-parentDMRmax']=allresultsegmentinfodf['PELTmax']-allresultsegmentinfodf['parentDMRmax']
allresultsegmentinfodf['PELTmin-parentDMRmin']=allresultsegmentinfodf['PELTmin']-allresultsegmentinfodf['parentDMRmin']



In [9]:
dmrchrindex=columnnames.index('DMRchrom')
foroutsubdf=indfdropped.iloc[:,dmrchrindex:]
foroutsubdf=foroutsubdf.reset_index( drop=True)
foroutsubdf=foroutsubdf.drop_duplicates(keep='first')
#foroutsubdf.head()

In [10]:
outdf=foroutsubdf.merge(allresultdf,how='inner',on=['DMRchrom','DMRstart','DMRend'])
outdf=outdf.rename(columns ={'DMRchrom':'chrom','DMRstart':'start','DMRend':'end'},errors='raise')
outdf.to_csv(outfile,sep="\t",index=False)


allresultsegmentinfodf.to_csv(outsegmentfile,sep="\t",index=False)